## 51. N-Queens

回溯法

In [ ]:
(defun solve-n-queens (n)
  (let ((solutions))
    (loop for col below n
       do (loop for item in (find-solution 0 n `(,col))
             do  (let ((solution (loop repeat n
                                    collect (loop repeat n
                                               collect "."))))
                   (loop for col in item
                      for row below (1- n)
                      do (setf (nth col (nth row solution)) "Q")
                      finally (push solution solutions))))
       finally (return solutions))))

(defun next-positions (n &optional placed)
  "对已顺序放置 placed 的 n 皇后问题，返回下一行可能的放置位置"
  (set-difference (loop for i below n collect i)
                  (loop for col in placed
                     for i from (length placed) downto 1
                     append `(,(- col i) ,col ,(+ col i)))))

(defun find-solution (row n &optional placed)
  "placed 是已放置的皇后，第 i 个元素表示 i 行 placed[i] 列放置了皇后，
   row 是当前行，placed[row] == placed[-1]。
   对于前面行已放置元素为 placed（长度为 row+1）的 n 皇后问题，找出所有解"
  (cond
    ((= row (- n 1))
     `(,placed))
    (t
     (loop for p in (next-positions n placed)
        append (find-solution
                (1+ row) n (append placed `(,p)))))))

## 52. N-Queens II

In [ ]:
(defun total-n-queens (n)
  (length
    (loop for col below n
       append (loop for item in (find-solution 0 n `(,col))
             collect item))))

(defun next-positions (n &optional placed)
  "对已顺序放置 placed 的 n 皇后问题，返回下一行可能的放置位置"
  (set-difference (loop for i below n collect i)
                  (loop for col in placed
                     for i from (length placed) downto 1
                     append `(,(- col i) ,col ,(+ col i)))))

(defun find-solution (row n &optional placed)
  "placed 是已放置的皇后，第 i 个元素表示 i 行 placed[i] 列放置了皇后，
   row 是当前行，placed[row] == placed[-1]。
   对于前面行已放置元素为 placed（长度为 row+1）的 n 皇后问题，找出所有解"
  (cond
    ((= row (- n 1))
     `(,placed))
    (t
     (loop for p in (next-positions n placed)
        append (find-solution
                (1+ row) n (append placed `(,p)))))))

## 53. Maximum Subarray

In [ ]:
(defun max-sub-array (nums)
  (let* ((lo 0)
         (hi (1- (length nums)))
         (sum (apply #'+ nums))
         (max sum))
    (if (> (apply #'max nums) max)
        (setf max (apply #'max nums)))
    (loop while (< lo hi)
       do (let* ((n-lo (next nums lo 1))
                 (sum1 (apply #'+ (subseq nums lo (1+ n-lo))))
                 (n-hi (next nums hi -1))
                 (sum2 (apply #'+ (subseq nums n-hi (1+ hi)))))
            (if (< sum1 sum2)
                (setf lo (1+ n-lo) sum (- sum sum1))
                (setf hi (1- n-hi) sum (- sum sum2)))
            (if (> sum max)
                (setf max sum)))
       finally (return max))))

(defun next (nums start step)
  "获取从 start 位置开始，第一个负数开始的连续负数序列中的最后一个的位置"
  (loop while (< -1 start (length nums))
     do (if (>= (nth start nums) 0)
            (incf start step)
            (return (loop while (and (< 0 (+ start step) (length nums))
                                     (< (nth (incf start step) nums) 0))
                       finally (return (- start step)))))
     finally (return (- start step))))

;;; Another amazing way
;;; https://leetcode.com/problems/maximum-subarray/discuss/20396/Easy-Python-Way
(defun max-sub-array (nums)
  (loop for i from 1 below (length nums)
     do (if (plusp (nth (1- i) nums))
            (incf (nth i nums) (nth (1- i) nums)))
     finally (return (apply #'max nums))))

## 54. Spiral Matrix

找规律。

假设开始点为 (x, y)，结束点为 (m, n)。

1. 向右：(x, y) -> (x, n)
2. 向下：(x+1, n) -> (m, n)
3. 向左：(m, n) -> (m, y)
4. 向上：(m, y) -> (x+1, y)
5. 汇聚值，x、y 加一，m、n 减一，重复步骤 1-5

In [ ]:
(defun spiral-order (matrix)
  (let ((m (1- (length matrix)))
        (n (1- (length (first matrix))))
        (x 0)
        (y 0))
    (loop while (and (<= x m)
                     (<= y n))
       append (append (loop for i from y to n
                         collect (nth i (nth x matrix)))
                      (loop for i from (1+ x) to m
                         collect (nth n (nth i matrix)))
                      (loop for i from (decf n) downto y
                         when (> m x)
                         collect (nth i (nth m matrix)))
                      (loop for i from (decf m) above x
                         collect (nth y (nth i matrix))))
       do (incf x)
         (incf y))))

## 55. Jump Game

跟之前做过的一道题类似，用表存储某个位置是否能到达最后。

$$canjump(0) = {\exists} \text{canjump(i)}, {i \le nums[0]} $$

In [ ]:
(defun can-jump (nums)
  (if (null nums) (return-from can-jump nil))
  (let* ((n (length nums))
         (table (loop repeat n collect nil)))
    (setf (nth (1- n) table) t)
    (loop for i from (- n 2) downto 0
       do (loop for j from 1 to (nth i nums)
             when (and (< (+ i j) n)
                       (nth (+ i j) table))
             do (return (setf (nth i table) t))))
    (first table)))

## 56. Merge Intervals

In [ ]:
(defun merge-i (intervals)
  (let* ((intervals (sort intervals #'(lambda (a b) (< (first a)
                                                       (first b)))))
         (res `(,(car intervals)))
         (intervals (cdr intervals)))
    (loop for interval in intervals
       do (if (>= (second (car (last res)))
                  (first interval))
              (setf (second (car (last res))) 
                    (max (second (first intervals))
                         (second (car (last res)))))
              (push interval (cdr (last res))))
       finally (return res))))

## 57. Insert Interval

In [ ]:
(defun insert-i (intervals new-interval)
  (let ((res `(nil)))
    (if (null intervals)
        (return-from insert-i `(,new-interval)))
    (loop for interval in intervals
       do (cond
            (new-interval
             (if (>= (second interval)
                     (first new-interval))
                 (progn
                   (push `(,(min (first interval) (first new-interval))
                            ,(max (second interval) (second new-interval)))
                         (cdr (last res)))
                   (setf new-interval nil))
                 (push interval (cdr (last res)))))
            (t
             (if (>= (second (car (last res)))
                     (first interval))
                 (setf (second (car (last res))) 
                       (max (second (first intervals))
                            (second (car (last res)))))
                 (push interval (cdr (last res))))))
       finally (return (cdr res)))))

## 58. Length of Last Word

In [ ]:
;; 使用正则
(ql:quickload "cl-ppcre")
(defun length-of-lastword (s)
  (length (car (last (ppcre:split " " s)))))

;; 使用循环
(defun length-of-lastword (s)
  (loop for c across (reverse (string-trim " " s))
     with res = 0
     do (if (string= " " c)
            (return res)
            (incf res))
     finally (return res)))

## 59. Spiral Matrix II

In [ ]:
(defun generate-matrix (n)
  (let ((x 0)
        (y 0)
        (index 0)
        (matrix (loop repeat n
                   collect (loop repeat n collect 0)))
        (n (1- n)))
    (loop while (and (<= x n)
                     (<= y n))
       do
         (loop for i from y to n
            do (setf (nth i (nth x matrix)) (incf index)))
         (loop for i from (1+ x) to n
            do (setf (nth n (nth i matrix)) (incf index)))
         (loop for i from (1- n) downto y
            when (> n x)
            do (setf (nth i (nth n matrix)) (incf index)))
         (loop for i from (decf n) above x
            do (setf (nth y (nth i matrix)) (incf index)))
         (incf x)
         (incf y)
       finally (return matrix))))